# Heat map of the current matrix 

In [3]:
import pyreadr
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import os 

current_directory = os.getcwd()

folder_name = "weight_tracking"  

directory_path = os.path.join(current_directory, folder_name)

file_count = len([name for name in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, name))])

# Function to load and plot heatmap for a given index
def plot_heatmap(file_index):
    rds_file_path = f"{directory_path}/S_curr_iteration_{file_index}.rds"
    
    
    rds_data = pyreadr.read_r(rds_file_path)
    
    # Extract the data from the dictionary
    df = None
    for key in rds_data.keys():
        df = rds_data[key]

    
    if not isinstance(df, pd.DataFrame):
        df = pd.DataFrame(df)

    # Create a custom diverging colormap centered at 0 
    cmap = sns.diverging_palette(240, 10, as_cmap=True)  

    
    plt.figure(figsize=(10, 8))
    sns.heatmap(df, cmap=cmap, center=0, cbar=True, xticklabels=False, yticklabels=False)
    plt.xlabel("")
    plt.ylabel("")
    plt.title(f"Heatmap of Weights for All X - Iteration {file_index}")
    plt.show()


slider = widgets.IntSlider(value=1, min=1, max=file_count, step=1, description='Iteration:')

# Link the slider to the plotting function
widgets.interact(plot_heatmap, file_index=slider)

# Display the slider
display(slider)


interactive(children=(IntSlider(value=1, description='Iteration:', max=1000, min=1), Output()), _dom_classes=(…

IntSlider(value=1, description='Iteration:', max=1000, min=1)

# Make a GIF to visualize the weight matrix evolution 

In [3]:
import pyreadr
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import os

current_directory = os.getcwd()

folder_name = "weight_tracking"  
directory_path = os.path.join(current_directory, folder_name)

# Create a 'gif' folder inside 'weight_tracking' to store the images
gif_folder_name = "gif"
gif_folder_path = os.path.join(directory_path, gif_folder_name)
os.makedirs(gif_folder_path, exist_ok=True)

file_count = len([name for name in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, name))])

images = []

# Generate and save each heatmap as an image
for i in range(1, file_count + 1):
    rds_file_path = f"{directory_path}/S_curr_iteration_{i}.rds"
    
    
    rds_data = pyreadr.read_r(rds_file_path)
    
    # Extract the data from the dictionary
    df = None
    for key in rds_data.keys():
        df = rds_data[key]

    
    if not isinstance(df, pd.DataFrame):
        df = pd.DataFrame(df)

    # Create a custom diverging colormap centered at 0
    cmap = sns.diverging_palette(240, 10, as_cmap=True)

    
    plt.figure(figsize=(10, 8))
    sns.heatmap(df, cmap=cmap, center=0, cbar=True, xticklabels=False, yticklabels=False)
    plt.xlabel("")
    plt.ylabel("")
    plt.title(f"Heatmap of Weights for All X - Iteration {i}")

    # Save the image in the 'gif' folder
    image_filename = os.path.join(gif_folder_path, f"heatmap_{i}.png")
    plt.savefig(image_filename)
    plt.close()
    
    images.append(image_filename)


output_gif_path = os.path.join(current_directory, "heatmap_animation.gif")

# Create the GIF
with imageio.get_writer(output_gif_path, mode='I', duration=0.5) as writer:
    for filename in images:
        image = imageio.imread(filename)
        writer.append_data(image)

print(f"GIF saved at {output_gif_path}")


C:\Users\alise\AppData\Local\Temp\ipykernel_20388\1740936097.py:62: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


GIF saved at h:\My Drive\Job\CausalInference\XGBoost_Smoother\heatmap_animation.gif


# Plotting individual weights across all boosting rounds 

In [2]:
import pyreadr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML
import os


sns.set(style="whitegrid")

urrent_directory = os.getcwd()

folder_name = "weight_tracking"  

directory_path = os.path.join(current_directory, folder_name)

file_count = len([name for name in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, name))])

# Load each matrix from the RDS files
matrices = []
for i in range(1, file_count + 1):
    rds_file_path = f"{directory_path}/S_curr_iteration_{i}.rds"
    rds_data = pyreadr.read_r(rds_file_path)
    
    matrix = None
    for key in rds_data.keys():
        matrix = rds_data[key]
        
    if isinstance(matrix, pd.DataFrame):
        matrix = matrix.to_numpy()

    matrices.append(matrix)

# Convert list of matrices to a 3D numpy array (num_files, rows, columns)
matrices = np.array(matrices)
rows, cols = matrices.shape[1], matrices.shape[2]

# Function to plot the selected matrix element across all files
def plot_matrix_element(row_index, col_index):
    element_values = matrices[:, row_index, col_index]  # Extract the selected element across all matrices
    
    plt.figure(figsize=(12, 7))
    sns.lineplot(x=range(1, file_count + 1), y=element_values, marker='o', color="teal")
    plt.title(f"Element [{row_index}, {col_index}] Across All Matrices", fontsize=16, fontweight='bold')
    plt.xlabel("Boosting Round", fontsize=14)
    plt.ylabel("Value", fontsize=14)
    plt.grid(True)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    sns.despine()  
    plt.show()

# Create interactive sliders
row_slider = widgets.IntSlider(value=0, min=0, max=rows-1, step=1, description='Row Index:',
                               style={'description_width': 'initial'}, layout=widgets.Layout(width='80%'))
col_slider = widgets.IntSlider(value=0, min=0, max=cols-1, step=1, description='Col Index:',
                               style={'description_width': 'initial'}, layout=widgets.Layout(width='80%'))

# Link the sliders to the plotting function
widgets.interact(plot_matrix_element, row_index=row_slider, col_index=col_slider)

# Display the sliders with some padding and styling
display(HTML("<style>.widget-label { font-size: 14px; font-weight: bold; } .widget-inline-hbox { padding: 10px; }</style>"))
display(row_slider, col_slider)

interactive(children=(IntSlider(value=0, description='Row Index:', layout=Layout(width='80%'), max=1000, style…

IntSlider(value=0, description='Row Index:', layout=Layout(width='80%'), max=1000, style=SliderStyle(descripti…

IntSlider(value=0, description='Col Index:', layout=Layout(width='80%'), max=1000, style=SliderStyle(descripti…